In [1]:
import h5py as h5
import hyperspy.api as hs
import numpy as np

import matplotlib.pyplot as plt
%matplotlib qt
plt.rcParams['font.family'] = 'serif'

import warnings
warnings.filterwarnings('ignore')


odemis_to_hyperspy() is a function which takes in a filename pointing to an Odemis CL .h5 file, and loads the signal into HyperSpy with the correct calibrations. The pxl_size refers to the pxl_size set when acquiring maps in Odemis, and needs to be set manually. It is therefore important to include this in the filename when storing the CL data.

In [2]:
def odemis_to_hyperspy(filename, pxl_size, save, specbin = 1):   

    f = h5.File(filename,'r')

    id_path = 'Acquisition2//ImageData/' # Path to the spectrum data in the h5 data tree
    x = f[id_path + 'Image'] # Data
    aq_title = f['Acquisition2/PhysicalData/ChannelDescription'].value[0].decode('utf-8') # Spectrum or Panchrom

    # Find which type of CL-data we have, spectrum or intensity
    if 'Spectrum' in aq_title:

        cltype = 'spectrum'

        # Strip unused dimensions and transpose/ reverse index order
        xx = x[:,0,0,:,:].transpose((2,1,0))

        # Interpolate data to linearize the wavelength scale
        w  = f[id_path + 'DimensionScaleC'].value *1e9 # wavelengths in nanometer
        wx = np.linspace(w.min(),w.max(),w.size) # create vector with evenly spaced wavelengths

        # Iterate through data, replace with interpolated point at linearly spaced wavelengths
        for i in np.arange(xx.shape[0]) :
            for j in np.arange(xx.shape[1]) :
                xx[i,j,:] = np.interp(wx, w, xx[i,j,:])

        # Wavelength slope and offset used for calibration
        wslope = wx[1] - wx[0]
        woffset = wx.min()

        # Create hyperspy Signal1D containing the data
        s = hs.signals.Signal1D(xx)

    elif 'CL intensity' in aq_title:

        cltype = 'panchrom'

        # strip unused dimensions and transpose/ reverse index order
        xx = x[0,0,0,:,:].transpose((1,0))

        s = hs.signals.Signal2D(xx)

    else:
        print('Unknown signal type')

    # Spatial calibration of axes
    s.metadata.General.title = aq_title
    s.metadata.General.original_filename = filename
    s.metadata.General.notes = cltype
    s.axes_manager[0].name = 'Position x'
    s.axes_manager[0].scale = pxl_size
    s.axes_manager[0].offset = f[id_path + 'XOffset'].value * 1e6
    s.axes_manager[0].units = 'nm'
    s.axes_manager[1].name = 'Position y'
    s.axes_manager[1].scale = pxl_size
    s.axes_manager[1].offset = f[id_path + 'YOffset'].value * 1e6
    s.axes_manager[1].units = 'nm'

    # Spectral calibration of axes
    if cltype == 'spectrum' :
        s.axes_manager[2].name = 'Wavelength'
        s.axes_manager[2].units = 'nm'
        s.axes_manager[2].offset = woffset
        s.axes_manager[2].scale = wslope
        s.metadata.signal_type = 'CL'

    f.close()

    # Rotate data 3 times
    for i in range(0,3):
        s.data = (np.rot90(s.data))
        temp2 = s.axes_manager[0].size
        s.axes_manager[0].size = s.axes_manager[1].size
        s.axes_manager[1].size = temp2

    # Flip data
    s.data = np.flip(s.data, axis = 1)

    s.change_dtype('float64')

    if (specbin > 1) and (cltype == 'spectrum'):
        s.rebin(scale=[1,1,specbin])
        
    s.save(save, extension='hdf5', overwrite = True)
    return s

Use the odemis_to_hyperspy( ) to convert and save the signal as a hdf5 file

In [3]:
filename = '20-02-17 CN1018 S249.07 Grid/CN1018 S249.07 grid NW1 150_800_40kX _5kV_30SI_ExTime100ms_Binn2_Gain1_Pxl30nm_Slit107um_.h5'

# Extract pixel size from filename (substring between '_Pxl' and 'nm_')
pxl_size = float(filename.split('_Pxl',1)[1].split('nm_', 1)[0])

# Convert to hyperspy signal with correct calibrations, and save 
s = odemis_to_hyperspy(filename = filename, pxl_size = pxl_size, save = 'hdf5/NW1')

print('Converted: ', s)

Converted:  <Signal1D, title: Spectrum, dimensions: (29, 27|1024)>


The hdf5 file can then be loaded and used for further processing

In [12]:
%matplotlib qt

# Load the hdf5 file, plot the data in the interactive window, and verify the calibration of the axes
s = hs.load('hdf5/NW1.hdf5')
s.plot()
s.axes_manager

<Axes manager, axes: (29, 27|1024)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
      Position x |     29 |      0 |   0.067 |      30 |     nm 
      Position y |     27 |      0 |    0.07 |      30 |     nm 
---------------- | ------ | ------ | ------- | ------- | ------ 
      Wavelength |   1024 |        | 2.3e+02 |    0.44 |     nm

In [23]:
# Plot map of UV peak intensity
s.isig[340.:380.].sum(axis=2).plot()

# Plot vertical line
s.inav[15,:].plot()
